<a href="https://colab.research.google.com/github/ijusplab/implantacao-automatica/blob/master/Implantacao_LOAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Criação de Planilha de Execução Semi-Automática para B-87 e B-88**

## **Visão Geral**

Este notebook utiliza dados do SisJEF, de súmulas de julgamento e do IBGE para gerar dados estruturados para implantação de benefícios assistenciais.

A tabela a seguir ilustra parcialmente a correspondência dos dados.

| SisJEF                       | Forma Final             |
| ---------------------------- | ----------------------- |
|`1_PROCESSO`                  | `PROCESSO`              |
|`2_POLO ATIVO`                | `TITULAR`               |
|`CIDADE DO POLO ATIVO`        | `MUNICIPIO_TITULAR`     |
|-                             | `CODIGO_IBGE`(**)       |
|`CPF CURADOR`                 | `CPF_TUTOR_CURADOR`     |
|`CPF REPRESENTANTE`           | `CPF_REPRESENTANTE`     |
|`CPF/CNPJ POLO ATIVO`         | `CPF_TITULAR`           |
|`DATA DO TRÂNSITO`            | `DATA_DO_TRANSITO`      |
|`ENDEREÇO DO POLO ATIVO`      | `ENDERECO_TITULAR`      |
|-                             | `CEP_TITULAR`           |
|`FÓRUM ATUAL DO PROCESSO`     | `FORUM`                 |
|`RESULTADO DA SENTENÇA`       | `RESULTADO_DA_SENTENCA` |
|`TUTOR/CURADOR DO POLO ATIVO` | `NOME_TUTOR_CURADOR`    |
|`VARA-GABINETE ATUAL`         | `VARA`                  |
|-                             | `ESPECIE`(*)            |
|-                             | `DIB`(*)                |
|-                             | `DIP`(*)                |
|-                             | `RMI`(*)                |
|-                             | `IMPLANTACAO_PADRAO`(*) |
|-                             | `RESTABELECIMENTO`(*)   |
|-                             | `NB`(*)                 |
**<small>(\*)</small> Informações extraídas da súmula do julgado.**
**<small>(\*\*)</small> Informações obtidas por meio do SIDRA/IBGE.**


## **Instruções**

**INSTRUÇÕES**

1) Execute a célula `CARREGAR OFÍCIO GENÉRICO EM LOTE`. 

2) Assim que o código for inicializado e aparecer o botão de `upload`, carregue o arquivo `txt` do ofício genérico em lote. Você verá uma prévia do conteúdo extraído logo abaixo da célula, em forma de tabela.

3) Execute a célula `CARREGAR SENTENÇAS`. 

4) Quando aparecer o botão de `upload`, carregue os arquivos `pdf` das sentenças proferidas no lote. Novamente uma prévia do conteúdo extraído aparecerá numa tabela logo abaixo da célula.

5) Execute a célula `MESCLAR DADOS` para reunir os dados do ofício genérico em lote com os dados extraídos das sentenças. 

6) Execute a célula `BAIXAR ARQUIVO CSV` para baixar os dados extraídos, já devidamente mesclados, em formato `csv`.

## **Ferramentas de Extração**

In [ ]:
#@title **CARREGAR OFÍCIO GENÉRICO EM LOTE** { vertical-output: true }
DOCUMENTO_SEI = "" #@param {type:"string"}
EXPEDIENTE_SEI = '0015738-61.2020.4.03.8001'

!pip install tika

from google.colab import files
from tika import parser
import pandas as pd
import re, os, requests, json
from IPython.display import display, HTML

##
## Para normalização dos nomes de colunas
##
class Utils:
  _instance = None

  def __init__(self):  
    self.__acentos = {
      'a': re.compile('[áàäãâ]'), 
      'e': re.compile('[éèëê]'), 
      'i': re.compile('[íìïî]'),
      'o': re.compile('[óòöõô]'),
      'u': re.compile('[úùüû]'),
      'c': re.compile('[ç]'),
      'A': re.compile('[ÁÀÄÃÂ]'), 
      'E': re.compile('[ÉÈËÊ]'), 
      'I': re.compile('[ÍÌÏÎ]'),
      'O': re.compile('[ÓÒÖÕÔ]'),
      'U': re.compile('[ÚÙÜÛ]'),
      'C': re.compile('[Ç]')
    }
    self.__caracteres_especiais = re.compile('[!@#$%¨&*+=ªº°§]')

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def remover_acentuacao(self, s):
    for letra, padrao in self.__acentos.items():
      s = re.sub(padrao, letra, s)
    return s

  def limpar(self, s):
    s = self.remover_acentuacao(s).upper().strip()
    s = re.sub(self.__caracteres_especiais, '', s).strip()
    s = re.sub(r'\s{2,}', ' ', s).strip()
    return s

  def normalizar(self, s):
    s = self.remover_acentuacao(s).upper().strip()
    s = re.sub(r'\d+_+', '', s)
    s = re.sub(r'\s+', '_', s)
    return s

  def ler_pdf(self, nome_arquivo):
    dados = parser.from_file(nome_arquivo)
    texto = dados['content']
    return str(texto)

##
## Instanciação
##
utils = Utils.instance()

##
## Extração do código IBGE dos municípios
##
class IBGE:
  _instance = None

  def __init__(self):  
    endpoint = 'https://servicodados.ibge.gov.br/api/v1/localidades/municipios'
    response = requests.get(endpoint)
    if response.status_code != 200:
      raise Exception(f'Erro ao tentar obter os dados do IBGE\sStatus Code: {response.status_code}')
    self.__dados_IBGE = response.json()

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def codigo_municipio(self, nome_municipio):
    for municipio in self.__dados_IBGE:
      if utils.normalizar(municipio['nome']) == utils.normalizar(nome_municipio):
        return municipio['id']
    return ''   

##
## Instanciação
##
ibge = IBGE.instance()

##
## Extração de dados de ofício genérico em lote
##
class OFGL:
  _instance = None

  def __init__(self):
    self.lote = ''
    self.data_frame = None
    self._cols = {
      "PROCESSO": "PROCESSO",
      "POLO_ATIVO": "TITULAR",
      "CIDADE_DO_POLO_ATIVO": "MUNICIPIO_TITULAR",
      "CODIGO_IBGE": "CODIGO_IBGE",
      "CPF_CURADOR": "CPF_TUTOR_CURADOR",
      "CPF_REPRESENTANTE": "CPF_REPRESENTANTE",
      "CPF/CNPJ_POLO_ATIVO": "CPF_TITULAR",
      "DATA_DO_TRANSITO": "DATA_DO_TRANSITO",
      "ENDERECO_DO_POLO_ATIVO": "ENDERECO_TITULAR",
      "FORUM_ATUAL_DO_PROCESSO": "FORUM",
      "RESULTADO_DA_SENTENCA": "RESULTADO_DA_SENTENCA",
      "TUTOR/CURADOR_DO_POLO_ATIVO": "NOME_TUTOR_CURADOR",
      "VARA-GABINETE_ATUAL": "VARA"
    }

  @classmethod
  def instance(cls):
      if cls._instance is None:
          cls._instance = cls()
      return cls._instance

  def __transformar_nome_coluna(self, s):
    if s in self._cols:
      return self._cols[s]
    return s

  def __transformar_cpf(self, val):
    # o dado pode vir com ou sem os pontos e o traço
    pattern = re.compile('^\d{3}\.?\d{3}\.?\d{3}-?\d{2}$')
    if not pattern.match(val.strip()):
      return val.strip()
    s = str(val).replace('.', '').replace('-', '')
    return f'{s[:3]}.{s[3:6]}.{s[6:9]}-{s[9:11]}'

  def __todos_iguais(self, digitos):
    return len(set(digitos)) == 1

  def __calcular_verificador(self, digitos):
    produto_soma = 0
    for i in range(len(digitos)):
      produto_soma += (digitos[i] * (len(digitos) + 1 - i))
    return (((produto_soma * 10) % 11) % 10)

  def __validar_cpf(self, val):
    if len(val) == 0:
      return val
    pattern = re.compile('^\d{3}\.\d{3}\.\d{3}-\d{2}$')
    if pattern.match(val):
      partes = str(val).split('-')
      digitos = [int(x) for x in partes[0].replace('.', '')]
      if self.__todos_iguais(digitos):
        return 'INVALIDO'
      verificadores = [int(x) for x in partes[1]]
      primeiro_verificador = self.__calcular_verificador(digitos)
      digitos.append(primeiro_verificador)
      segundo_verificador = self.__calcular_verificador(digitos)
      if primeiro_verificador == verificadores[0] and segundo_verificador == verificadores[1]:
        return val
    return 'INVALIDO'

  def __transformar_data(self, val):
    pattern = re.compile('\d{1,2}[\/.\-]\d{1,2}[\/.\-]\d{2,4}')
    match = pattern.search(val)
    if match:
      return match.group(0).replace('.', '/').replace('-', '/')
    return val
  
  def __extrair_cep(self, val):
    pattern = re.compile('\s+CEP:\s*(\d{5}\-\d{0,3})')
    match = pattern.search(val)
    if match:
      return match.group(1)
    return ''

  def __transformar_endereco(self, val):
    pattern = re.compile('\s+CEP:\s*(\d{5}\-\d{0,3})')
    return re.sub(pattern, '', val)

  def __transformar_resultado(self, val):
    pattern = re.compile('PROCEDENTE EM PARTE|PROCEDENTE|ACORDO')
    match = pattern.search(val)
    if match:
      return match.group(0)
    return val
  
  def __transformar_nome_curador(self, val):
    return '\n'.join(val.split(' / ')).strip()

  def load(self):
    arquivos = files.upload()
    if len(arquivos) > 0:
      for nome, conteudo in arquivos.items():
        if re.match('^.+\.(txt)$', nome):
          # o parâmetro dtype=str mantém todos os dados como strings, evitando que
          # o pandas faça conversões indesejadas, como, por exemplo, cpf para
          # número, tirando os zeros à esquerda.
          df = pd.read_csv(nome, sep=';', dtype=str, encoding='latin1')
          df = df.fillna('')
          df.columns = map(utils.normalizar, df.columns)
          df.columns = map(self.__transformar_nome_coluna, df.columns)
          codigo_ibge = df['MUNICIPIO_TITULAR'].apply(ibge.codigo_municipio)
          df.insert(3, 'CODIGO_IBGE', codigo_ibge, True)
          df['TITULAR'] = df['TITULAR'].apply(utils.limpar)
          df['MUNICIPIO_TITULAR'] = df['MUNICIPIO_TITULAR'].apply(utils.limpar)
          df['CPF_TUTOR_CURADOR'] = df['CPF_TUTOR_CURADOR'].apply(self.__transformar_cpf)
          df['CPF_REPRESENTANTE'] = df['CPF_REPRESENTANTE'].apply(self.__transformar_cpf)
          df['CPF_TITULAR'] = df['CPF_TITULAR'].apply(self.__transformar_cpf)
          df['CPF_TUTOR_CURADOR'] = df['CPF_TUTOR_CURADOR'].apply(self.__validar_cpf)
          df['CPF_REPRESENTANTE'] = df['CPF_REPRESENTANTE'].apply(self.__validar_cpf)
          df['CPF_TITULAR'] = df['CPF_TITULAR'].apply(self.__validar_cpf)
          # CPF do titular não pode estar em branco
          df['CPF_TITULAR'] = df['CPF_TITULAR'].apply(lambda x: x if len(x) > 0 else 'INVALIDO')
          df['DATA_DO_TRANSITO'] = df['DATA_DO_TRANSITO'].apply(self.__transformar_data)
          cep = df['ENDERECO_TITULAR'].apply(self.__extrair_cep)
          df['ENDERECO_TITULAR'] = df['ENDERECO_TITULAR'].apply(utils.limpar).apply(self.__transformar_endereco)
          df.insert(9, 'CEP_TITULAR', cep, True)
          df['FORUM'] = df['FORUM'].apply(utils.limpar)
          df['RESULTADO_DA_SENTENCA'] = df['RESULTADO_DA_SENTENCA'].apply(self.__transformar_resultado)
          df['NOME_TUTOR_CURADOR'] = df['NOME_TUTOR_CURADOR'].apply(utils.limpar).apply(self.__transformar_nome_curador)
          df['VARA'] = df['VARA'].apply(utils.limpar)
          df['EXPEDIENTE_SEI'] = range(len(df))
          df['EXPEDIENTE_SEI'] = df['EXPEDIENTE_SEI'].apply(lambda x: EXPEDIENTE_SEI)
          df['DOCUMENTO_SEI'] = range(len(df))
          df['DOCUMENTO_SEI'] = df['DOCUMENTO_SEI'].apply(lambda x: DOCUMENTO_SEI)

          self.__lote = nome
          self.__data_frame = df
        else:
          print('>>>> ATENÇÃO')
          print('>>>> O arquivo ' + nome + ' não é um txt')

  def has_data_frame(self):
    return not self.__data_frame is None

  def get_name(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    return self.__lote

  def get_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    return self.__data_frame

  def show_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ LOTE CARREGADO')
      return
    print('LOTE:')
    display(HTML(self.__data_frame.to_html()))

##
## Instanciação
##
ofgl = OFGL.instance()
ofgl.load()
ofgl.show_data_frame()

In [ ]:
#@title **CARREGAR SENTENÇAS** { vertical-output: true }

##
## Extração de dados de sentenças
##

class Sentencas:
  def __init__(self):
    # O número do processo é o único elemento não extraído da súmula, mas do cabeçalho
    # da sentença. Assim, mesmo os processos que não tenham súmula podem ser identificados.
    self.__re_numero_processo = re.compile('PROCESSO Nr:\s*?(\d{7}\-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4})', re.DOTALL)
    # É importante que a súmula tenha um formato rigoroso e destacado, para assegurar
    # que as informações sejam extraídas apenas dali e não da fundamentação da sentença
    # (a fundamentação pode trazer citações ou referências que digam respeito a outros
    # processos ou a outros benefícios)
    self.__re_sumula = re.compile('\*{40,}([^*]+)\*{40,}', re.DOTALL)
    self.__padroes = {
        'ESPECIE': re.compile('ESP[ÉE]CIE DO NB:\s([À-ü]?.*)RMI', re.DOTALL ),
        'RESTABELECIMENTO': re.compile('ESP[ÉE]CIE DO NB:\s*[À-ü]?.*(RESTABELECIMENTO)', re.DOTALL | re.IGNORECASE),
        'DIB': re.compile('DIB:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL),
        'DIP': re.compile('DIP:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL),
        'DCB': re.compile('DCB:\s*?(\d{1,2}[/\-.]\d{1,2}[/\-.]\d{2,4})', re.DOTALL),
        'NB_OBJETO_ACAO': re.compile('NB:\s*(\d{10})', re.DOTALL),
        'RMI': re.compile('RMI:\s*R\$\s*([\d.,]+)', re.DOTALL),
        'CURADOR_PROVISORIO': re.compile('CURADOR PROVISÓRIO:\s*[À-ü]?.*(SIM)', re.DOTALL | re.IGNORECASE),
        'IMPLANTACAO_PADRAO': re.compile('IMPLANTAÇÃO-PADRÃO:\s*[À-ü]?.*(N[AÃ]O)', re.DOTALL | re.IGNORECASE)
    }
    self.__sentencas = []
    self.__data_frame = None

  def __extrair_numero_processo(self, texto):
    match = self.__re_numero_processo.search(texto)
    if match:
      return match.group(1)
    return ''

  def __extrair_sumula(self, texto):
    match = self.__re_sumula.search(texto)
    if match:
      return match.group(1)
    return ''

  def __transformar_especie(self, val):
    if val == 'INVALIDO':
      return ''
    return 88 if re.compile('IDOS[AO]|88').search(val.upper()) else 87

  def __transformar_data(self, val):
    if val == 'INVALIDO':
      return ''
    val = val.replace('.', '/').replace('-', '/')
    if (val == '00/00/0000'):
      return ''
    return val

  def __transformar_rmi(self, val):
    if val == 'INVALIDO':
      return ''
    return ''

  def __transformar_booleano(self, val):
    if val == 'INVALIDO':
      return val
    return 'SIM' if len(val) > 0 else 'NAO'

  def __transformar_implantacao_padrao(self, val):
    return 'SIM' if len(val) == 0 else 'NAO'

  def __criar_coluna(self, rotulo, padrao):
    linhas = []
    for sentenca in self.__sentencas:
      if sentenca['sumula']:
        match = padrao.search(sentenca['sumula'])
        if match:
          linhas.append(utils.limpar(match.group(1)))
        else:
          linhas.append('')
          print('>>>> Atributo "' + rotulo + '" não encontrado em ' + sentenca['nome'])
      else:
        print('>>>> Súmula não encontrada em ' + sentenca['nome'])
        linhas.append('INVALIDO')
    return linhas

  def __criar_coluna(self, rotulo, padrao):
    linhas = []
    for sentenca in self.__sentencas:
      if sentenca['sumula']:
        match = padrao.search(sentenca['sumula'])
        if match:
          linhas.append(match.group(1))
        else:
          linhas.append('')
          print('>>>> Atributo "' + rotulo + '" não encontrado em ' + sentenca['nome'])
      else:
        print('>>>> Súmula não encontrada em ' + sentenca['nome'])
        linhas.append('INVALIDO')
    return linhas

  def create_data_frame(self):
    if len(self.__sentencas) > 0:
      colunas = ['PROCESSO'] + list(self.__padroes.keys())
      dados = {}
      processos = []
      for sentenca in self.__sentencas:
        processos.append(sentenca['processo'])
      dados['PROCESSO'] = processos
      for rotulo, padrao in self.__padroes.items():
        dados[rotulo] = self.__criar_coluna(rotulo, padrao)
      df = pd.DataFrame(dados, columns=colunas)
      df = df.fillna('')
      df['ESPECIE'] = df['ESPECIE'].apply(self.__transformar_especie)
      df['RESTABELECIMENTO'] = df['RESTABELECIMENTO'].apply(self.__transformar_booleano)
      df['DIB'] = df['DIB'].apply(self.__transformar_data)
      df['DIP'] = df['DIP'].apply(self.__transformar_data)
      df['DCB'] = df['DCB'].apply(self.__transformar_data)
      df['RMI'] = df['RMI'].apply(self.__transformar_rmi)
      df['CURADOR_PROVISORIO'] = df['CURADOR_PROVISORIO'].apply(self.__transformar_booleano)
      df['IMPLANTACAO_PADRAO'] = df['IMPLANTACAO_PADRAO'].apply(self.__transformar_implantacao_padrao)
      self.__data_frame = df

  def load(self):
    arquivos = files.upload()
    if len(arquivos) > 0:
      for nome, conteudo in arquivos.items():
        if re.match('^.+\.(pdf)$', nome):
          conteudo = utils.ler_pdf(nome)
          self.__sentencas.append({
              'nome': nome,
              'conteudo': conteudo,
              'processo': self.__extrair_numero_processo(conteudo),
              'sumula': self.__extrair_sumula(conteudo)
          })
        else:
          print('>>>> ATENÇÃO')
          print('>>>> O arquivo ' + nome + ' não é um pdf')

  def has_data_frame(self):
    return not self.__data_frame is None

  def get_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ SENTENÇAS CARREGADAS')
      return
    return self.__data_frame

  def show_data_frame(self):
    if not self.has_data_frame():
      print('>>>> NÃO HÁ SENTENÇAS CARREGADAS')
      return
    print('SENTENÇAS:')
    display(HTML(self.__data_frame.to_html()))

##
## Instanciação
##
sentencas = Sentencas()
sentencas.load()
sentencas.create_data_frame()
sentencas.show_data_frame()

In [ ]:
#@title **MESCLAR DADOS** { vertical-output: true }

##
## Para criar e baixar csv único
##

import csv
import numpy as np

class CsvBuilder:
  def __init__(self):
    self.__data_frame = None
    self.__name = ''

  def __is_invalid(self, row):
    for val in row:
      if val == 'INVALIDO':
        return True
    return False

  def load_data_frame(self, df, column):
    if self.__data_frame is None:
      self.__data_frame = df
    else:
      self.__data_frame = self.__data_frame.merge(df, on=column) 

  def set_name(self, name):
    self.__name = name
  
  def validate_rows(self):
    if self.__data_frame is None:
      print('>>>> NÃO HÁ DADOS CARREGADOS')
      return
    for index, row in self.__data_frame.iterrows():
      nb = row['NB_OBJETO_ACAO'] if (row['RESTABELECIMENTO'] == 'SIM') else ''
      if (row['RESTABELECIMENTO'] == 'SIM' and nb):
        nb == 'INVALIDO'
      implantacao = 'NAO' if (row['IMPLANTACAO_PADRAO'] == 'NAO') | self.__is_invalid(row) else 'SIM'
      self.__data_frame.at[index, 'NB_OBJETO_ACAO'] = nb
      self.__data_frame.at[index, 'IMPLANTACAO_PADRAO'] = implantacao

  def show_data_frame(self):
    if self.__data_frame is None:
      print('>>>> NÃO HÁ DADOS CARREGADOS')
      return
    if len(self.__name) == 0:
      print('>>>> NOME DO ARQUIVO NÃO FORNECIDO')
      return
    print(f'DADOS MESCLADOS ({self.__name}):')
    display(HTML(self.__data_frame.to_html()))

  def download(self):
    if self.__data_frame is None:
      print('>>>> NÃO HÁ DADOS CARREGADOS')
      return
    with open(f'{self.__name}.csv', 'w', encoding='latin1') as file:
      # especificações do INSS: sem índice, delimitado por ";" e sem aspas 
      file.write(self.__data_frame.to_csv(index=False, sep=';', quoting=csv.QUOTE_NONE))
    files.download(f'{self.__name}.csv')

##
## Instanciação
##
builder = CsvBuilder()
builder.load_data_frame(ofgl.get_data_frame(), 'PROCESSO')
builder.load_data_frame(sentencas.get_data_frame(), 'PROCESSO')
builder.validate_rows()
builder.set_name(ofgl.get_name())
builder.show_data_frame()


In [ ]:
#@title **BAIXAR ARQUIVO `CSV`**

builder.download()